In [1]:
!pip install confluent-kafka

     |████████████████████████████████| 8.1 MB 23.2 MB/s eta 0:00:01


In [2]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [3]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [4]:
conf = {'bootstrap.servers': bootstrap_server}

In [5]:
a = AdminClient(conf)

In [6]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 1 topics:
  "_schemas" with 1 partition(s)


In [7]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

In [8]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
import time

topic = "TemperatureSensorEvent"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [9]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

## run the following cell to demostrate that fire is not detected

In [10]:
import json
from random import gauss

while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(20, 1.0),"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


{'sensor': 'S1', 'temperature': 21.731105455964585, 'ts': 1606232071}
{'sensor': 'S1', 'temperature': 20.45618833625183, 'ts': 1606232082}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 0
{'sensor': 'S1', 'temperature': 19.913187558860802, 'ts': 1606232092}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 1
{'sensor': 'S1', 'temperature': 20.81828476345006, 'ts': 1606232102}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 2
{'sensor': 'S1', 'temperature': 20.054285720118937, 'ts': 1606232112}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 3
{'sensor': 'S1', 'temperature': 18.6608704823843, 'ts': 1606232122}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 4
{'sensor': 'S1', 'temperature': 19.709396657769005, 'ts': 1606232132}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 5
{'sensor': 'S1', 'temperature': 20.28286479862641, 'ts': 1606232142}
Produce

KeyboardInterrupt: 

## run the following cell to demostrate to detect fire

In [ ]:
while True:
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(55, 1.0),"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)

{'sensor': 'S1', 'temperature': 54.831176555315075, 'ts': 1606232670}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 59
{'sensor': 'S1', 'temperature': 54.32476408364215, 'ts': 1606232680}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 60
{'sensor': 'S1', 'temperature': 56.08683922464601, 'ts': 1606232690}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 61
{'sensor': 'S1', 'temperature': 54.16334995583129, 'ts': 1606232700}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 62
{'sensor': 'S1', 'temperature': 55.826471157258936, 'ts': 1606232710}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 63
{'sensor': 'S1', 'temperature': 55.833924512225245, 'ts': 1606232720}
Produced record to topic TemperatureSensorEvent partition [0] @ offset 64
{'sensor': 'S1', 'temperature': 55.71221114579938, 'ts': 1606232730}
Produced record to topic TemperatureSensorEvent partition [0] @ offset